In [1]:
%load_ext autoreload
%autoreload 2
from gn4pions.modules.nn_functions import * 

In [2]:
df = load_data(n_files=100)

100%|██████████| 100/100 [02:09<00:00,  1.30s/it]


Dataframe has 1,027,650 events.


/clusterfs/ml4hep/mpettee/miniconda3/envs/nbdev/lib/python3.9/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [3]:
df.keys()

Index(['cluster_e_0', 'cluster_calib_e_0', 'cluster_had_weight_0', 'track_pt',
       'track_eta', 'track_phi', 'track_z0', 'truth_particle_e',
       'truth_particle_pt', 'log10_cluster_e_0', 'log10_cluster_calib_e_0',
       'log10_cluster_had_weight_0', 'log10_track_pt',
       'log10_truth_particle_e', 'log10_truth_particle_pt'],
      dtype='object')

In [4]:
# # recreate EM
# train_vars = ['log10_cluster_e_0',]
# target_var = ['log10_cluster_calib_e_0']

In [5]:
# recreate track resolution
train_vars = ['log10_track_pt', 'track_eta']
target_var = ['log10_truth_particle_e']

In [ ]:
test = train(df, train_vars, target_var, epochs=20)

Epoch 1/20
19725/19725 [==============================] - 44s 2ms/step - loss: 0.0330 - val_loss: 0.0326
Epoch 2/20
19725/19725 [==============================] - 44s 2ms/step - loss: 0.0273 - val_loss: 0.0289
Epoch 3/20
16482/19725 [========================>.....] - ETA: 6s - loss: 0.0271

In [ ]:
make_plots(test, plot_em=False, plot_track=True)